Laboratorio 1



Andre Marroquin Tarot- 22266

Sergio Orellana- 221122

Rodrigo Mansilla 22611


LINK GIT: https://github.com/mar22266/LABORATORIOS-IA.git

Task 1
1. Considera un modelo de regresión lineal con dos características, X₁ y X₂, y sus pesos correspondientes w₁ y w₂. Si el modelo predice una salida y mediante la ecuación y = 2w₁X₁ + 3w₂X₂ + 1, ¿cuál es la interpretación del coeficiente 3w₂ en el contexto del modelo?

R//

2. Explica el concepto de multicolinealidad en el contexto de la regresión lineal. ¿Cómo afecta la multicolinealidad a la interpretación de los coeficientes de regresión individuales? 

R//

Task 2

In [ ]:
#s

Task 2.1

In [ ]:
#r

Task 2.2

In [ ]:
#d

Referencias: